## Web scrapping de IMDB

In [18]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

# Download IMDB's Top 250 data
url = 'http://www.imdb.com/chart/top'


,movie_title,year,place,star_cast,rating,vote,link
0,Cadena perpetua,1994,1,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.220086937930727,None,/title/tt0111161/
1,El padrino,1972,2,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.147355884254592,None,/title/tt0068646/
2,El padrino: Parte II,1974,3,"Francis Ford Coppola (dir.), Al Pacino, Robert...",8.980081220514458,None,/title/tt0071562/
3,El caballero oscuro,2008,4,"Christopher Nolan (dir.), Christian Bale, Heat...",8.970803734877,None,/title/tt0468569/
4,12 hombres sin piedad,1957,5,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",8.936241387241546,None,/title/tt0050083/
...,...,...,...,...,...,...,...
245,Fanny y Alexander,1982,246,"Ingmar Bergman (dir.), Bertil Guve, Pernilla A...",8.021987108222715,None,/title/tt0083922/
246,Neon Genesis Evangelion: The End of Evangelion,1997,247,"Hideaki Anno (dir.), Megumi Ogata, Megumi Haya...",8.021553071161161,None,/title/tt0169858/
247,Drishyam,2013,248,"Jeethu Joseph (dir.), Mohanlal, Meena",8.021135627267235,None,/title/tt3417422/
248,Soul,2020,249,"Pete Docter (dir.), Jamie Foxx, Tina Fey",8.021023506607449,None,/title/tt2948372/


In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

# Download IMDB's Top 250 data
url = 'http://www.imdb.com/chart/top'
response = requests.get(url)
soup = bs(response.text, 'lxml')



In [26]:
movies = soup.select("td.titleColumn") #('td', class_='titleColumn')
movies[0]

<td class="titleColumn">
      1.
      <a href="/title/tt0111161/" title="Frank Darabont (dir.), Tim Robbins, Morgan Freeman">Cadena perpetua</a>
<span class="secondaryInfo">(1994)</span>
</td>

In [17]:
link = soup.select('td.titleColumn a')
links =[a.attrs.get('href') for a in link] # [a['href'] for a in link]
'https://www.imdb.com'+links[0]

'https://www.imdb.com/title/tt0111161/'

In [13]:
crew = soup.select('td.titleColumn a')
crews = [a.attrs.get('title') for a in crew]
crews[0]

'Frank Darabont (dir.), Tim Robbins, Morgan Freeman'

In [19]:
rating = soup.select("td.posterColumn span[name=ir]")
ratings = [b.attrs.get('data-value') for b in rating]
ratings[0]

'9.23985412378404'

In [22]:
vote = soup.select("td.ratingColumn strong")
votes = [b.attrs.get('data-value') for b in vote]
type(votes[0])

NoneType

In [39]:
movie_string = movies[0].get_text()
movie_string

'\n      1.\n      Cadena perpetua\n(1994)\n'

In [3]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd


url = 'http://www.imdb.com/chart/top'
response = requests.get(url)
soup = bs(response.text, 'lxml')

movies = soup.select("td.titleColumn")
link = soup.select('td.titleColumn a')
links =[a.attrs.get('href') for a in link]
crew = soup.select('td.titleColumn a')
crews = [a.attrs.get('title') for a in crew]
rating = soup.select("td.posterColumn span[name=ir]")
ratings = [b.attrs.get('data-value') for b in rating]
vote = soup.select("td.ratingColumn strong")
votes = [b.attrs.get('data-value') for b in vote]



imdb = []

for index in range(0,len(movies)):
    movie_string = movies[index].get_text()
    movie = (' ').join(movie_string.split()).replace('.',',') # 100, cadena perpetua (1994)
    movie_title = movie[len(str(index))+1:-7]
    year = re.search('\((.*?)\)', movie_string).group(1)
    place = movie[:len(str(index))-(len(movie))]

    data = {"movie_title": movie_title,
            "year": year,
            "place": place,
            "star_cast": crews[index],
            "rating": ratings[index],
            "vote": votes[index],
            "link": links[index]}
    
    imdb.append(data)

df = pd.DataFrame(imdb)
df
    
    

,movie_title,year,place,star_cast,rating,vote,link
0,Cadena perpetua,1994,1,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.239856258618092,None,/title/tt0111161/
1,El padrino,1972,2,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.160737544918108,None,/title/tt0068646/
2,El caballero oscuro,2008,3,"Christopher Nolan (dir.), Christian Bale, Heat...",8.992193010863582,None,/title/tt0468569/
3,El padrino: Parte II,1974,4,"Francis Ford Coppola (dir.), Al Pacino, Robert...",8.9899820371598,None,/title/tt0071562/
4,12 hombres sin piedad,1957,5,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",8.950328875940587,None,/title/tt0050083/
...,...,...,...,...,...,...,...
245,La bella y la bestia,1991,246,"Gary Trousdale (dir.), Paige O'Hara, Robby Benson",8.017477946534775,None,/title/tt0101414/
246,Gandhi,1982,247,"Richard Attenborough (dir.), Ben Kingsley, Joh...",8.015616729970288,None,/title/tt0083987/
247,Criadas y señoras,2011,248,"Tate Taylor (dir.), Emma Stone, Viola Davis",8.01482280406514,None,/title/tt1454029/
248,La doncella,2016,249,"Park Chan-wook (dir.), Kim Min-hee, Ha Jung-woo",8.012687350410939,None,/title/tt4016934/
